## Using a planner

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.0.1"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.0.1-preview"
#r "nuget: Microsoft.SemanticKernel.Connectors.OpenAI, 1.0.1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.0.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.0.1-alpha"


Installed Packages Microsoft.SemanticKernel, 1.0.1 Microsoft.SemanticKernel.Connectors.OpenAI, 1.0.1 Microsoft.SemanticKernel.Planners.Handlebars, 1.0.1-preview Microsoft.SemanticKernel.Plugins.Core, 1.0.1-alpha Microsoft.SemanticKernel.Plugins.Web, 1.0.1-alpha

In [2]:
#!import ../config/Settings.cs
#!import ../Plugins/EmailPlugin.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Planning.Handlebars;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Plugins.Core;
using Microsoft.SemanticKernel.Plugins.Web;
using Microsoft.SemanticKernel.Plugins.Web.Bing;
using System.IO;

var (useAzureOpenAI, model, azureEndpoint, apiKey, bingApiKey, orgId) = Settings.LoadFromFile();
var kernel = Microsoft.SemanticKernel.Kernel.CreateBuilder()
            .AddAzureOpenAIChatCompletion(
                model,   // deployment name
                azureEndpoint, // Azure OpenAI Endpoint
                apiKey)      // Azure OpenAI Key
            .Build();

kernel.Plugins.Clear();
var pluginPath = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "..", "Plugins");
foreach (var plugin in new string[] { "WeatherPlugin", "WriterPlugin", "TranslatePlugin" })
{
    Console.WriteLine($"Loading ${pluginPath}${plugin}");
    kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginPath, plugin));
}
#pragma warning disable SKEXP0050
kernel.ImportPluginFromType<TimePlugin>();   
#pragma warning disable SKEXP0054
var bing = kernel.ImportPluginFromType<SearchUrlPlugin>("search");
// #pragma warning disable SKEXP0054
// var bingConnector = new BingConnector(bingApiKey);
// var bing = new WebSearchEnginePlugin(bingConnector);
// kernel.ImportPluginFromObject(bing, "bing");

var kernelPlugin = KernelPluginFactory.CreateFromFunctions(
    "AddressBookPlugin",
    "Manages properties of people with email addresses",
    new KernelFunction[] { 
            kernel.CreateFunctionFromMethod(
                ([Description("Person name")]string name) => $"{name}@contoso.com",
                functionName: "GetEmailAddress",
                description: "Given person name, return the person's email address"),
            kernel.CreateFunctionFromMethod(
                ([Description("Text to output")]string text) => Console.WriteLine(text),
                functionName: "DisplayText",
                description: "Output completion results to the screen")                
    }
);
kernel.Plugins.Add(kernelPlugin);


Loading $c:\Users\mrochon\source\repos\ai-samples\NotebookSK\4. Plan\..\Plugins$WeatherPlugin
Loading $c:\Users\mrochon\source\repos\ai-samples\NotebookSK\4. Plan\..\Plugins$WriterPlugin
Loading $c:\Users\mrochon\source\repos\ai-samples\NotebookSK\4. Plan\..\Plugins$TranslatePlugin


List plugins and functions

In [ ]:
    foreach (KernelFunctionMetadata func in kernel.Plugins.GetFunctionsMetadata())
    {
        Console.WriteLine($"Plugin: {func.PluginName}");
        Console.WriteLine($"   {func.Name}: {func.Description}");

        if (func.Parameters.Count > 0)
        {
            Console.WriteLine("      Params:");
            foreach (var p in func.Parameters)
            {
                Console.WriteLine($"      - {p.Name}: {p.Description}");
                Console.WriteLine($"        default: '{p.DefaultValue}'");
            }
        }

        Console.WriteLine();
    }

Now try different prompts

In [8]:
#pragma warning disable SKEXP0060
    var planner = new HandlebarsPlanner() { AllowLoops = true });
    //var plan = await planner.CreatePlanAsync(kernel, "Check the weather in Seattle, write an email, in Danish to Jane with dressing tips based on the results");
    //var goal = "Check the weather in Seattle, find Jane's email address and write her an email with some dressing tips based on the results";    
    //var goal = "How do I search for weather in Seattle?";
    var goal = "Use web search to find Seattle weather";
    var plan = await planner.CreatePlanAsync(kernel, goal);
    var resp = await plan.InvokeAsync(kernel);
    Console.WriteLine("Plan:\n");
    Console.WriteLine(plan.ToString());
    Console.WriteLine("Result:\n");
    Console.WriteLine(resp);

    // Should be able to save and re-load a plan: https://devblogs.microsoft.com/semantic-kernel/migrating-from-the-sequential-and-stepwise-planners-to-the-new-handlebars-and-stepwise-planner/

Plan:

{{!-- Step 1: Create search URL for Seattle weather --}}
{{set "searchUrl" (WeatherPlugin-Simple city="Seattle")}}

{{!-- Step 2: Search for Seattle weather and get the result --}}
{{set "searchResult" (search-BingSearchUrl query=searchUrl)}}

{{!-- Step 3: Print the search result to the screen --}}
{{json (concat "The weather in Seattle is: " searchResult)}}
Result:

The weather in Seattle is: https://www.bing.com/search?q=city%3A%20Seattle%0AsearchUrl%3A%20https%3A%2F%2Fwww.msn.com%2Fen-us%2Fweather%2Fforecast%2Fin-Seattle,WA


You can now modify, load and execute the plan.